# Washington Group 1, 2 and 3 Preliminary Analysis
## 20 August 2020

As part of the Fall preparation sprint, we spent the last ten days analyzing what it will take to beat the infection decisively this fall. There are three fundamental ideas:
1. Backstop. Have a backstop that protects hospitals in the unlikely event, they outstrip their ability to procure the right resources. They are our last line of defense against the infection.
2. Purchasing Cooperative. For smaller groups that have funds, but who can benefit from the aggregation of supply. This will protect an important group that is upstream of hospitals.
3. Protecting the most vulnerable. The infection starts at this front line. We need to defend the most vulnerable against the infection. Not only does this reduce the disease burden, it also increases employment and takes the pressure off our relief efforts. Winning upstream, winning at the front lines, that is the key to victory.

Questions and edits to rich@restart.us and lucas@restart.us

In [1]:
import sys
import pandas as pd
sys.path.append('../src')
from restart import NoteCompose
from util import set_config, to_df, to_sheet, display_population

# Group 1 Backstop to Large Providers
These are the large hospitals and other facilities that take care of our sickest patients. With the fall, we will face the difficult challenge of both the flu season and the recurrence of COVID-19 as the climate worsens and people move indoors.

This group will provide most of their own PPE, disinfection and other resources with the state acting as a backstop in extreme emergencies. Given the exponential nature of infection, we need this backstop because even the best predictions have a large variance. For example, if the disease doubles every week, then even a two week error in estimate will increase PPE requirements in COVID wards by 4x.

# Group 2 Aggregate Demand of Smaller Providers

This second group consists of several different populations

- Smaller health care providers such as hospitals with 299 beds or less
- long-term care providers and nursing homes
- behavioral health facilities
- dentists
- morticians
- Federally Qualified Health Centers [FQHC](https://www.hrsa.gov/opa/eligibility-and-registration/health-centers/fqhc/index.html) community health providers
- public health organizations,
- tribal clinics
- independent physician practices
- First responders including EMOs, police, fire

We use two different methods to estimate these populations. More detailed surveys and census methods are also possible, so consider these methods as ways to get a broad measure of the scope of the problem. 

## Employee Classification (SOC) Analysis

SOC codes starting with "29-", "31-", and "33-" refer to, respectively, healthcare occupations, healthcare support occupations, and protection services. We estimate of the percentage of healthcare workers or healthcare-support workers fall into the Group 2 category, and simply scale all the numbers by that amount. It seems like a safe assumption that all the protection services would fall into this group. 

This analysis provides the stockpile that you would need to 100% cover the group for 30 days. A key policy decision is the size of the back stop needed. If you want to cover 50% of the demand for 30 days, then the figures would be half that.


In [2]:
import numpy as np
import ipysheet
import ipywidgets as widgets
from ipywidgets import Layout

def set_stock(days):
    resource.demand(resource.inventory_ln_df)
    resource.set_inv_min(demand.level_total_demand_ln_df, days)
    stockpile = to_sheet(resource.inventory_ln_df)
    stockpile_cost_df = resource.inventory_ln_df.multiply(resource.cost_ln_df).round()
    stockpile_cost = to_sheet(stockpile_cost_df)
    display_population(stockpile)
    display_population(stockpile_cost, money=True)
    population_out = to_sheet(population.detail_pd_df)
    display_population(population_out)

In [3]:
config = set_config('../src')
restart = NoteCompose(configdir='../src/', population='oes', demand='washington', subpop='wa_tier2_opt1', state='Washington')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand

widgets.interact(set_stock, days=30)

ValueError: Shape of passed values is (87, 7), indices imply (87, 2)

In [ ]:
essential_arr = restart.model.demand.level_pl_df.to_numpy()
population_labels = list(restart.model.demand.level_pl_df.index)

essential_list = []

for row in essential_arr:
    if row[0] == 1:
        essential_list.append('Y')
    else:
        essential_list.append('N')

essential_df = pd.DataFrame(essential_list)
essential_df.insert(loc=0, column='Population', value=population_labels)
essential_df.columns = ['Population', 'Essential Worker']
essential_sheet = to_sheet(essential_df)
essential_sheet.row_headers = False

essential_sheet

### Note on specific SOC codes used in Analysis

The main assumption here is that we are going to include certain groups as part of the coding for Group 2 this would include, a subset of occupations in these groups. Note that there is some overlap here because this is a list of all occupations and some percentage of these work in Group 1. These can of course all be modified, but this is the basis for the "quick" pull done here:

| OCC Category  | SOC Code |
| :- | -: |
| OCC Category 29 | SOC Code |
| Dental Hygienists | 29-1292 |   
| EMTs and Paramedics | 29-2040 |
| Family Medicine Physicians | 29-1215 | 
| Respiratory Therapists | 29-1126 |
| Psychiatrists | 29-1223 |
| Audiologists | 29-1181 |
| Pediatricians | 29-1221 |
| Psychiatric Technicians | 29-2052 |

| OCC Category 31 | SOC Code |
| :- | -: |
| Home Health and Personal Care Aides | 31-1120 |
| Nursing Assistants | 31-1131   |
| Morticians, Undertakers, and Funeral Arrangers |  39-4031 |
| Orderlies | 31-1132 |
| Psychiatric Aides | 31-1133  |
| Dental Assistants | 31-9091 |

| OCC Category 33 and 39 | SOC Code |
| :- | -: |
| Firefighters |  33-2011 |
| Correctional Officers and Jailers |  33-3012 |
| Detectives and Criminal Investigators |  33-3021  |
| Transportation Security Screeners | 33-9093 |
| Parking Enforcement Workers | 33-3041   |
| Police and Sheriff’s Patrol officers | 33-3051 |
| Transit and Railroad Police | 33-3052 |
| Embalmers |  39-4011 |


In [ ]:
restart = NoteCompose(configdir='../src/', population='oes', demand='washington', subpop='wa_tier2_opt2', state='Washington')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand

widgets.interact(set_stock, days=30)

# Vulnerable Populations 

These figures are very preliminary and are derived by initial census data gathered by Group 3. Overall they cover these populations (with caveats noted):

- Aging and Long-term Support. This is the total count of Long Term Care and also aging. We have another figure for just Long Term care of 120K, so there are some differences here. Also some of these may be able to pay.
- Developmentally Disabled. Some of the staff is captured in the Group 2 figures as there are job classes here that overlap and we will have to figure out which belong in Group 2 and which in Group 3 (eg not enough funds to pay)
- Low Income and Poverty. This is the largest figure of clients with nealry 1.8M. The staff does not count social assistance groups that are providing support here so that staff is under counted.
- Child Communites and Staff. This is a subset of school children
- Behavioral Health. We also have some overlap here in the staff with Group 2 that we need to exclude.
- Agricultural Community. This is mainly workers and some have materials and others do not. Also there are other programs that may cover them.

In [ ]:
restart = NoteCompose(configdir='../config/vuln', population='dict', demand='dict')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand

widgets.interact(set_stock, days=30)